In [1]:
import numpy as np
import cv2 as cv
import glob
import csv
import matplotlib.pyplot as plt

# Write frames as images

In [6]:
# This section reads a video file and writes every second as a png image titled frameXXXcamXXX.png

cap = cv.VideoCapture("cam5.MOV") # Update Here
if not cap.isOpened():
    print("Error opening video file")
    exit()

frame_number = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame_number += 1
    if frame_number % 60 == 0:
        if not cv.imwrite(
            r"C:\Users\darren\Desktop\BR2-Camera-Calibration-Workspace\cam distortion params script\frame"
            + str(int(frame_number / 60))
            + "cam5.png", # Update Here
            frame,
        ):
            print("image could not be saved.")

# Perform Calibration

In [19]:
# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((7 * 9, 3), np.float32)
objp[:, :2] = np.mgrid[0:9, 0:7].T.reshape(-1, 2)
# Arrays to store object points and image points from all the images.
objpoints = []  # 3d point in real world space
imgpoints = []  # 2d points in image plane.
images = glob.glob("*cam5.png")   # Update Here
for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, (9, 7), None)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
        imgpoints.append(corners2)

ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(
    objpoints, imgpoints, gray.shape[::-1], None, None
)

In [21]:
csv_file_path = (
    r"C:\Users\darren\Desktop\BR2-Camera-Calibration-Workspace\cam distortion params script\csv\7.20.23cam5.csv" # Update Here
)

with open(csv_file_path, mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerows(mtx)
    writer.writerows(dist)

# Re-projection Error


In [ ]:
mean_error = 0
data = []
for i in range(len(objpoints)):
    imgpoints2, _ = cv.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv.norm(imgpoints[i], imgpoints2, cv.NORM_L2) / len(imgpoints2)
    data.append(error * len(imgpoints2))
    mean_error += error
print("total error: {}".format(mean_error / len(objpoints)))

plt.hist(data, bins="auto")
plt.xlabel("distance in pixels")
plt.ylabel("Frequency")
plt.title("Histogram of distances between distorted and undistorted points")
plt.show()

In [ ]:
# testing code

In [14]:
import cv2

frame = cv2.imread("./frame27cam1.png")
h, w = frame.shape[:2]
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w, h), 1, (w, h))
# undistort
dst = cv2.undistort(frame, mtx, dist, None, newcameramtx)
# crop the image
x, y, w, h = roi
dst = dst[y : y + h, x : x + w]

cv2.imshow("degas", dst)
cv2.waitKey(0)
cv2.destroyAllWindows()